# 2.7 Model Export & Deployment

This notebook finalizes the Heart Disease UCI model by rebuilding a full preprocessing + model inference pipeline and exporting it for deployment. It reproduces evaluation metrics (Accuracy ≈ 0.88, F1 ≈ 0.89, ROC AUC ≈ 0.905) for the **threshold-tuned SVC** selected in the previous hyperparameter tuning stage.

**Key Outputs**
- Deployed pipeline (`../models/best_model.pkl`)
- Model report (`../results/best_model_report.json`)
- Verification summary (final cell)

> All steps are deterministic with `random_state=42` for reproducibility.

## 1. Imports & Reproducibility Setup
Load core libraries and ensure directories for models/results exist. We re-instantiate the chosen SVC with the tuned hyperparameters and will later apply the selected decision threshold.

In [47]:
# Imports & reproducibility setup
import os, json, joblib, warnings, time
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
warnings.filterwarnings('ignore')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Paths
ROOT = Path('..')
DATA_PATH = ROOT / 'data' / 'selected_features.csv'
MODELS_DIR = ROOT / 'models'
RESULTS_DIR = ROOT / 'results'
MODELS_DIR.mkdir(exist_ok=True, parents=True)
RESULTS_DIR.mkdir(exist_ok=True, parents=True)

# Attempt to hydrate BEST_PARAMS & threshold from prior tuning artifacts if present
REPORT_JSON_PATH = RESULTS_DIR / 'best_model_report.json'
DEFAULT_BEST_PARAMS = {
    'C': 1.5,          # fallback tuned C (will be overridden if report exists)
    'kernel': 'rbf',
    'gamma': 'scale',  # fallback
    'probability': True,
    'class_weight': None,
    'random_state': RANDOM_STATE
}
DEFAULT_THRESHOLD = 0.42

if REPORT_JSON_PATH.exists():
    try:
        with open(REPORT_JSON_PATH, 'r', encoding='utf-8') as f:
            _rep = json.load(f)
        if _rep.get('model_name') == 'Threshold-Tuned SVC':
            BEST_PARAMS = _rep.get('best_params', DEFAULT_BEST_PARAMS)
            SELECTED_THRESHOLD = _rep.get('best_threshold', DEFAULT_THRESHOLD)
        else:
            BEST_PARAMS = DEFAULT_BEST_PARAMS
            SELECTED_THRESHOLD = DEFAULT_THRESHOLD
    except Exception as e:
        print('[WARN] Failed to read existing report, using defaults:', e)
        BEST_PARAMS = DEFAULT_BEST_PARAMS
        SELECTED_THRESHOLD = DEFAULT_THRESHOLD
else:
    BEST_PARAMS = DEFAULT_BEST_PARAMS
    SELECTED_THRESHOLD = DEFAULT_THRESHOLD

print('Configured Threshold-Tuned SVC best_params:', BEST_PARAMS)
print('Selected decision threshold:', SELECTED_THRESHOLD)

Configured Threshold-Tuned SVC best_params: {'C': 1.5, 'kernel': 'rbf', 'gamma': 'scale', 'probability': True, 'class_weight': None, 'random_state': 42}
Selected decision threshold: 0.617


## 2. Load Cleaned Dataset
Load the engineered, feature-selected dataset `selected_features.csv` from `../data/`. The target column is inferred (`target` or last column).

In [48]:
# Load dataset
if not DATA_PATH.exists():
    raise FileNotFoundError(f'Missing dataset: {DATA_PATH}. Ensure preprocessing notebook exported selected_features.csv')

df = pd.read_csv(DATA_PATH)
print('Loaded dataset:', DATA_PATH, 'shape=', df.shape)
TARGET_COL = 'target' if 'target' in df.columns else df.columns[-1]
X = df.drop(columns=[TARGET_COL]).copy()
y = df[TARGET_COL].copy()
print('Feature columns:', len(X.columns))
print('Target distribution:', y.value_counts(normalize=True).round(3).to_dict())

Loaded dataset: ..\data\selected_features.csv shape= (920, 18)
Feature columns: 17
Feature columns: 17
Target distribution: {1: 0.553, 0: 0.447}
Target distribution: {1: 0.553, 0: 0.447}


## 3. Train/Test Split
Hold out 20% of the data for final evaluation. Stratified split preserves class balance.

In [49]:
from sklearn.model_selection import train_test_split
strat = y if y.nunique() > 1 else None
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=strat
)
print('Train:', X_train.shape, 'Test:', X_test.shape)

Train: (736, 17) Test: (184, 17)


## 4. Rebuild Preprocessing + Threshold-Tuned SVC Pipeline
We:
1. Identify numeric vs categorical columns.
2. Apply scaling to numeric features and one-hot encode categoricals.
3. Fit the SVC with tuned hyperparameters on training data.
4. Apply a custom prediction helper that enforces the selected decision threshold.

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, ClassifierMixin

# Split feature types
numeric_cols = X_train.select_dtypes(include=['int64','int32','float64','float32']).columns.tolist()
categorical_cols = [c for c in X_train.columns if c not in numeric_cols]
print(f'Numeric cols: {len(numeric_cols)} | Categorical cols: {len(categorical_cols)}')

numeric_pipeline = Pipeline([
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_cols),
    ('cat', cat_pipeline, categorical_cols)
])

# Threshold wrapper
class ThresholdSVC(BaseEstimator, ClassifierMixin):
    def __init__(self, threshold=0.5, **svc_params):
        self.threshold = threshold
        self.svc_params = svc_params
        self.model_ = SVC(**svc_params)
    def fit(self, X, y):
        self.model_.fit(X, y)
        return self
    def predict(self, X):
        if hasattr(self.model_, 'predict_proba'):
            prob = self.model_.predict_proba(X)[:,1]
        else:
            scores = self.model_.decision_function(X)
            mn, mx = scores.min(), scores.max()
            prob = (scores - mn)/(mx - mn + 1e-9)
        return (prob >= self.threshold).astype(int)
    def predict_proba(self, X):
        if hasattr(self.model_, 'predict_proba'):
            return self.model_.predict_proba(X)
        scores = self.model_.decision_function(X)
        mn, mx = scores.min(), scores.max()
        prob_pos = (scores - mn)/(mx - mn + 1e-9)
        prob = np.vstack([1 - prob_pos, prob_pos]).T
        return prob
    def get_params(self, deep=True):
        return {'threshold': self.threshold, **self.svc_params}
    def set_params(self, **params):
        if 'threshold' in params:
            self.threshold = params.pop('threshold')
        self.svc_params.update(params)
        self.model_.set_params(**self.svc_params)
        return self

svc_core = ThresholdSVC(threshold=SELECTED_THRESHOLD, **BEST_PARAMS)

final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('svc', svc_core)
])

final_pipeline.fit(X_train, y_train)
print('Pipeline fitted.')

Numeric cols: 8 | Categorical cols: 9
Pipeline fitted.
Pipeline fitted.


## 5. Evaluation
Compute performance metrics on the held-out test split and verify they align with prior tuning (Accuracy ≈ 0.88, F1 ≈ 0.89, ROC AUC ≈ 0.905).

In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Predictions using thresholded pipeline
y_pred = final_pipeline.predict(X_test)
# For AUC need probabilities (post-preprocessor). Using predict_proba wrapper
proba = final_pipeline.predict_proba(X_test)
prob_pos = proba[:,1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
roc_auc = roc_auc_score(y_test, prob_pos)

metrics = {
    'accuracy': round(float(accuracy), 4),
    'precision': round(float(precision), 4),
    'recall': round(float(recall), 4),
    'f1': round(float(f1), 4),
    'roc_auc': round(float(roc_auc), 4)
}
print('Computed metrics (threshold applied):', metrics)

# Target (reference) metrics for sanity (tolerance allowed due to randomness / splits)
TARGET_METRICS = {'accuracy': 0.8804, 'f1': 0.8932, 'roc_auc': 0.9052}
TOL = 0.015
print('\nReference check:')
for k, v in TARGET_METRICS.items():
    diff = metrics[k] - v
    status = 'OK' if abs(diff) <= TOL else 'DRIFT'
    print(f" {k}: got {metrics[k]:.4f} target {v:.4f} Δ={diff:+.4f} [{status}]")

metrics['status'] = {k: ('OK' if abs(metrics[k]-TARGET_METRICS[k])<=TOL else 'DRIFT') for k in TARGET_METRICS}

Computed metrics (threshold applied): {'accuracy': 0.875, 'precision': 0.8911, 'recall': 0.8824, 'f1': 0.8867, 'roc_auc': 0.9063}

Reference check:
 accuracy: got 0.8750 target 0.8804 Δ=-0.0054 [OK]
 f1: got 0.8867 target 0.8932 Δ=-0.0065 [OK]
 roc_auc: got 0.9063 target 0.9052 Δ=+0.0011 [OK]


## 6. Save Exported Pipeline & Report
Persist the full preprocessing + SVC pipeline and a JSON report containing model metadata, threshold, metrics, and reproducibility info.

In [52]:
# Save pipeline and report (force sync with threshold 0.617 & current metrics)
MODEL_EXPORT_PATH = MODELS_DIR / 'best_model.pkl'
REPORT_PATH = RESULTS_DIR / 'best_model_report.json'

joblib.dump(final_pipeline, MODEL_EXPORT_PATH)
print('Saved pipeline ->', MODEL_EXPORT_PATH)

report = {
    'model_name': 'Threshold-Tuned SVC',
    'best_params': BEST_PARAMS,
    'best_threshold': SELECTED_THRESHOLD,
    'metrics': {k: metrics[k] for k in ['accuracy','precision','recall','f1','roc_auc']},
    'random_state': RANDOM_STATE,
    'cv_strategy': '5-Fold StratifiedKFold',
    'preprocessing': {
        'numeric_scaling': 'StandardScaler',
        'categorical_encoding': 'OneHotEncoder(ignore_unknown)',
        'numeric_features': numeric_cols,
        'categorical_features': categorical_cols
    },
    'dataset_used': str(DATA_PATH.name),
    'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
    'status': metrics.get('status', {})
}
with open(REPORT_PATH, 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2)
print('Saved report ->', REPORT_PATH)

# Display report preview
print('\nReport snapshot:')
for k,v in report['metrics'].items():
    print(f" {k}: {v}")

Saved pipeline -> ..\models\best_model.pkl
Saved report -> ..\results\best_model_report.json

Report snapshot:
 accuracy: 0.875
 precision: 0.8911
 recall: 0.8824
 f1: 0.8867
 roc_auc: 0.9063

Report snapshot:
 accuracy: 0.875
 precision: 0.8911
 recall: 0.8824
 f1: 0.8867
 roc_auc: 0.9063


## 7. Validation (Reload & Re-Evaluate)
Reload the exported pipeline (`best_model.pkl`), run predictions with the embedded threshold logic, and confirm metrics match expected tuned performance (≈ Accuracy 0.8804, F1 0.8932, ROC AUC 0.9052).

In [53]:
# Reload and validate
reloaded = joblib.load(MODEL_EXPORT_PATH)
re_y_pred = reloaded.predict(X_test)
re_proba = reloaded.predict_proba(X_test)[:,1]
re_metrics = {
    'accuracy': round(accuracy_score(y_test, re_y_pred), 4),
    'precision': round(precision_score(y_test, re_y_pred, zero_division=0), 4),
    'recall': round(recall_score(y_test, re_y_pred, zero_division=0), 4),
    'f1': round(f1_score(y_test, re_y_pred, zero_division=0), 4),
    'roc_auc': round(roc_auc_score(y_test, re_proba), 4)
}
print('Reloaded metrics:', re_metrics)

# Load report to cross-check
with open(REPORT_PATH, 'r', encoding='utf-8') as f:
    saved_report = json.load(f)
rep_metrics = saved_report.get('metrics', {})

# Consistency check
print('\nConsistency check (reloaded vs report):')
for k in ['accuracy','precision','recall','f1','roc_auc']:
    rv = re_metrics[k]; sv = rep_metrics.get(k)
    delta = rv - sv if sv is not None else float('nan')
    status = 'MATCH' if sv is not None and abs(delta) <= 0.002 else 'MISMATCH'
    print(f" {k}: reload={rv:.4f} report={sv:.4f} Δ={delta:+.4f} [{status}]")

print('='*30)
print('✅ Final Model Exported')
print('='*30)
print('Model: Threshold-Tuned SVC')
print(f"Accuracy: {re_metrics['accuracy']:.4f}")
print(f"F1 Score: {re_metrics['f1']:.4f}")
print(f"ROC AUC: {re_metrics['roc_auc']:.4f}")
print(f"Decision Threshold: {SELECTED_THRESHOLD}")
print('Params source: best_model_report.json' if (RESULTS_DIR / 'best_model_report.json').exists() else 'Params source: defaults (report missing)')
print(f'Saved at: {MODEL_EXPORT_PATH}')
print(f'Report at: {REPORT_PATH}')
print('='*30)

Reloaded metrics: {'accuracy': 0.875, 'precision': 0.8911, 'recall': 0.8824, 'f1': 0.8867, 'roc_auc': 0.9063}

Consistency check (reloaded vs report):
 accuracy: reload=0.8750 report=0.8750 Δ=+0.0000 [MATCH]
 precision: reload=0.8911 report=0.8911 Δ=+0.0000 [MATCH]
 recall: reload=0.8824 report=0.8824 Δ=+0.0000 [MATCH]
 f1: reload=0.8867 report=0.8867 Δ=+0.0000 [MATCH]
 roc_auc: reload=0.9063 report=0.9063 Δ=+0.0000 [MATCH]
✅ Final Model Exported
Model: Threshold-Tuned SVC
Accuracy: 0.8750
F1 Score: 0.8867
ROC AUC: 0.9063
Decision Threshold: 0.617
Params source: best_model_report.json
Saved at: ..\models\best_model.pkl
Report at: ..\results\best_model_report.json
